In [1]:
from ortools.linear_solver import pywraplp
import math


def read_data(file_name):
    data = {}

    with open(file_name, 'r') as f:
        if f == None:
            print('File not found')
            return None

        data["num_subjects"], data["num_rooms"] = [int(x) for x in f.readline().split()]
        data["num_students_per_subject"] = [int(x) for x in f.readline().split()]
        data["num_seats_per_room"] = [int(x) for x in f.readline().split()]
        data["num_pairs"] = int(f.readline())
        data["conflicts"] = []
        for _ in range(data["num_pairs"]):
            s1, s2 = [int(x) for x in f.readline().split()]
            data["conflicts"].append((s1 - 1, s2 - 1))

    return data



STATUS_DICT = {
    pywraplp.Solver.OPTIMAL: "OPTIMAL",
    pywraplp.Solver.FEASIBLE: "FEASIBLE",
    pywraplp.Solver.INFEASIBLE: "INFEASIBLE",
    pywraplp.Solver.UNBOUNDED: "UNBOUNDED",
    pywraplp.Solver.ABNORMAL: "ABNORMAL",
    pywraplp.Solver.NOT_SOLVED: "NOT_SOLVED",
}


def solve_with_mip(num_subjects: int, num_rooms: int, nums_student_per_subject: int,
                   num_seats_per_room: int, subject_pairs: int,
                   num_sections_per_day: int = 4,
                   ):

    max_sum_sections = num_subjects

    # Create a new linear solver
    solver = pywraplp.Solver('Scheduling', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    x = {}

    for subject in range(num_subjects):
        for room in range(num_rooms):
            for section_id in range(max_sum_sections):
                x[(subject, room, section_id)] = solver.IntVar(0, 1, 'x[%i, %i, %i]' % (subject, room, section_id))

    y = solver.IntVar(0, max_sum_sections, 'y')

    # Constraints 1: Each subject must be assigned once
    for subject in range(num_subjects):
        temp = []

        for room in range(num_rooms):
            c = solver.Sum([x[(subject, room, section_id)] for section_id in range(max_sum_sections)])
            temp.append(c)

        solver.Add(solver.Sum(temp) == 1)

        # Constraints 2: Each room must have most one subject in each section
    for room in range(num_rooms):
        for section_id in range(max_sum_sections):
            solver.Add(solver.Sum([x[(subject, room, section_id)]
                                   for subject in range(num_subjects)]) <= 1)

    # Constraints 3: Each subject assigned to a room must have enough seats
    for subject in range(num_subjects):
        for room in range(num_rooms):
            solver.Add(solver.Sum([x[(subject, room, section_id)] * nums_student_per_subject[subject]
                                   for section_id in range(max_sum_sections)]) <= num_seats_per_room[room])

    # Constraints 4: Two subjects cannot be assigned to the same room in the same section
    for section_id in range(max_sum_sections):
        for subject1, subject2 in subject_pairs:
            solver.Add(solver.Sum([x[(subject1, room, section_id)] + x[(subject2, room, section_id)]
                                   for room in range(num_rooms)]) <= 1)

    # Constraints 5: Minimize the number of sections
    for subject in range(num_subjects):
        for room in range(num_rooms):
            for section_id in range(max_sum_sections):
                solver.Add(y >= x[(subject, room, section_id)] * section_id)

    # Objective function
    solver.Minimize(y)

    # Solve the problem
    status = solver.Solve()

    # print('Objective value =', solver.Objective().Value())
    for subject in range(num_subjects):
        for room in range(num_rooms):
            for section_id in range(max_sum_sections):
                if x[(subject, room, section_id)].solution_value() > 0:
                    print(f"{subject + 1} {section_id + 1} {room + 1}")

    num_days = (solver.Objective().Value() % num_sections_per_day)

    return num_days


In [2]:
import os
import time
import pandas as pd

test_cases = []
times = []
objective_values = []


for test_case in ['data_10_2_(0).txt', 'data_10_2_(1).txt', 'data_10_2_(2).txt', 
                  'data_16_3_(0).txt',  'data_16_3_(1).txt', 'data_16_3_(2).txt',
                  'data_20_4_(0).txt', 'data_20_4_(1).txt', 'data_20_4_(2).txt',
                  ]:

    print(f"Start solving {test_case}")
    start_time = time.time()
    data = read_data("../data/" + test_case)
    day = solve_with_mip(
        data["num_subjects"],
        data["num_rooms"],
        data["num_students_per_subject"],
        data["num_seats_per_room"],
        data["conflicts"],
    )
    end_time = time.time()
    test_cases.append(test_case)
    times.append(end_time - start_time)
    objective_values.append(day)
    print(f"Finished solving {test_case}")
    print(f"Time: {end_time - start_time}")

result = pd.DataFrame({
    "test_case": test_cases,
    "time": times,
    "objective_value": objective_values
})

Start solving data_10_2_(0).txt
1 1 2
2 5 2
3 3 2
4 2 1
5 2 2
6 5 1
7 4 1
8 1 1
9 4 2
10 3 1
Finished solving data_10_2_(0).txt
Time: 1.6880342960357666
Start solving data_10_2_(1).txt
1 1 2
2 5 1
3 1 1
4 9 1
5 2 1
6 3 1
7 6 1
8 4 1
9 7 1
10 8 1
Finished solving data_10_2_(1).txt
Time: 0.3859875202178955
Start solving data_10_2_(2).txt
1 5 1
2 4 1
3 8 1
4 7 1
5 3 1
6 2 1
7 7 2
8 8 2
9 6 1
10 1 1
Finished solving data_10_2_(2).txt
Time: 0.6990108489990234
Start solving data_16_3_(0).txt
1 3 1
2 3 2
3 5 2
4 1 1
5 6 3
6 6 2
7 2 2
8 1 3
9 3 3
10 2 1
11 5 1
12 5 3
13 4 2
14 2 3
15 1 2
16 4 3
Finished solving data_16_3_(0).txt
Time: 7.300976753234863
Start solving data_16_3_(1).txt
1 6 2
2 4 2
3 1 2
4 5 1
5 3 2
6 5 2
7 9 1
8 2 2
9 5 3
10 8 2
11 10 2
12 1 1
13 11 3
14 11 2
15 7 2
16 9 2
Finished solving data_16_3_(1).txt
Time: 6.910051107406616
Start solving data_16_3_(2).txt
1 1 1
2 1 2
3 2 1
4 5 2
5 6 3
6 3 3
7 6 1
8 4 2
9 2 2
10 6 2
11 3 2
12 5 3
13 4 1
14 5 1
15 3 1
16 2 3
Finished solvin